In [1]:
from os import listdir
from os.path import isfile, isdir, join

In [2]:
train_path = "/data/Plant_Clef/train"

files = {
    f.split(".")[0]: {
        "image": join(train_path, f),
        "xml": join(train_path, f.split(".")[0]+".xml")
    } 
    for f in listdir(train_path)
    if f[0] != "." and f[-3:] == "jpg"
}

print(len(files))

91758


In [3]:
import xml.etree.ElementTree as ET
from tqdm import tqdm


for f in tqdm(files):
    xml = files[f]['xml']
    tree = ET.parse(xml)
    root = tree.getroot()
    data = {}
    for child in root:
        data[child.tag] = child.text
    files[f]['data'] = data

100%|██████████| 91758/91758 [07:32<00:00, 202.91it/s]


In [4]:
from pprint import pprint

pprint(files['9733'])

{'data': {'Author': 'daniel barthelemy',
          'ClassId': '4736',
          'Content': 'Leaf',
          'Date': '2013-11-3',
          'Family': 'Rosaceae',
          'Genus': 'Cydonia',
          'ImageId2014': '36922',
          'Latitude': '43.13079',
          'LearnTag': 'Train',
          'Location': 'Toulon',
          'Longitude': '5.9022',
          'MediaId': '9733',
          'ObservationId': '13689',
          'ObservationId2014': '7746',
          'Species': 'Cydonia oblonga Mill.',
          'Vote': '4',
          'YearInCLEF': 'PlantCLEF2014'},
 'image': '/data/Plant_Clef/train/9733.jpg',
 'xml': '/data/Plant_Clef/train/9733.xml'}


In [6]:
import json

with open("plant_clef_data.json", "w") as F:
    json.dump(files, F)

# Import Data to TensorFlow

In [59]:
import tensorflow as tf
import tensorflow_addons as tfa

In [56]:
img_files = []
labels    = []

for f in files:
    img_files.append(files[f]['image'])
    labels.append(files[f]['data']['Content'])
    
IMG_WIDTH = 150
IMG_HEIGHT = 150
BATCH_SIZE = 64

Map = {}
count = 0
Labels = []

for label in labels:
    if not Map.get(label):
        Map[label] = count
        count += 1
    Labels.append(Map[label])
    

In [18]:
def decode_img(img):
    # convert the compressed string to a 3D uint8 tensor
    img = tf.image.decode_jpeg(img, channels=3)
    # Use `convert_image_dtype` to convert to floats in the [0,1] range.
    img = tf.image.convert_image_dtype(img, tf.float32)
    # resize the image to the desired size.
    return tf.image.resize(img, [IMG_WIDTH, IMG_HEIGHT])

In [19]:
def process_path(file_path):
    # load the raw data from the file as a string
    img = tf.io.read_file(file_path)
    img = decode_img(img)
    return img

In [20]:
list_ds = tf.data.Dataset.from_tensor_slices(img_files)

In [27]:
image_ds = list_ds.map(process_path)

In [30]:
label_ds = tf.data.Dataset.from_tensor_slices(Labels)

In [62]:
train_ds = tf.data.Dataset.zip((image_ds, label_ds))

train_ds = train_ds.shuffle(1024).batch(BATCH_SIZE)

# Model Creation

In [66]:
model = tf.keras.Sequential()

model.add(tf.keras.Input(shape=(IMG_WIDTH, IMG_HEIGHT, 3)))

model.add(tf.keras.layers.Conv2D(
    64, 
    (2,2), 
    strides=2,
    activation='relu',
    padding='same'
))

model.add(tf.keras.layers.Conv2D(
    64, 
    (2,2), 
    strides=3,
    activation='relu',
    padding='same'
))

model.add(tf.keras.layers.Conv2D(
    64, 
    (2,2), 
    strides=2,
    activation='relu',
    padding='same'
))

model.add(tf.keras.layers.Flatten())

model.add(tf.keras.layers.Dense(256, activation=None))

model.add(tf.keras.layers.Lambda(lambda x: tf.math.l2_normalize(x, axis=1)) # L2 normalize embeddings
)

model.summary()

Model: "sequential_24"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_37 (Conv2D)           (None, 75, 75, 64)        832       
_________________________________________________________________
conv2d_38 (Conv2D)           (None, 25, 25, 64)        16448     
_________________________________________________________________
conv2d_39 (Conv2D)           (None, 13, 13, 64)        16448     
_________________________________________________________________
flatten_7 (Flatten)          (None, 10816)             0         
_________________________________________________________________
dense_3 (Dense)              (None, 256)               2769152   
_________________________________________________________________
lambda_1 (Lambda)            (None, 256)               0         
Total params: 2,802,880
Trainable params: 2,802,880
Non-trainable params: 0
___________________________________________

In [67]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(0.001),
    loss=tfa.losses.TripletSemiHardLoss())

In [ ]:
# Train the network
history = model.fit(
    train_ds,
    epochs=10)

Epoch 1/10
    184/Unknown - 2818s 15s/step - loss: 0.9088